In [13]:
!pip install pandas

In [14]:
import random
import pandas as pd
from operator import itemgetter

In [15]:
# Download the "vehicle_data.csv" and upload that file to the local directory before running this section

# Step 1: Read vehicle emissions data from static csv database into a nested dictionary for computation
def csv_to_nested_dict_pandas(filename):
    df = pd.read_csv(filename)
    return df.groupby('vehicle_model').apply(lambda x: x.to_dict(orient='records')).to_dict()


vehicle_data_dict = csv_to_nested_dict_pandas('/content/vehicle_data.csv')

In [16]:
# Step 2: Define "mock" google maps api function to generates a random distance
def google_maps_distance_km():
    return round(random.random() * 50, 2)

In [17]:
# Step 3: Define "mock" google maps api function to calculate the time taken for different vehicle types to travel
def google_maps_time_taken(input_distance, vehicle_model):
    return round(input_distance / vehicle_data_dict[vehicle_model][0]["speed_km/hr"], 2)

In [18]:
# Step 4: Define user profile. Relevant variables are the car_model, which will be fixed as the same throughout this algorithm, and the CO2 quota remaining.
user_profile = {
    "user_car_model": "Toyota Corolla", # i am just assuming its just this car model bc headache
    "CO2_quota_remaining": 50,
    "weekly_CO2_quota_set": 50
}

In [19]:
# Step 5: Ask user for input starting and destination locations and generate a distance with our mock google maps api
location_1 = input("Insert starting location: ")
location_2 = input("Insert destination: ")
distance = google_maps_distance_km()

Insert starting location: Geylang
Insert destination: Woodlands


In [20]:
# Step 6: Define function to calculate total carbon emissions for given duration of journey and vehicle used
def emissions_per_trip(time_taken, vehicle_model):
    return round(time_taken * vehicle_data_dict[vehicle_model][0]["emissions_per_hour_kg"], 1)

In [21]:
# Step 7: Calculate the time_taken and emissions for each vehicle to travel the given distance, tabulating the data into a nested dictionary
route_data_dict = {
    "Car": {
        "time_taken": google_maps_time_taken(distance, user_profile["user_car_model"]),
        "emissions": emissions_per_trip(google_maps_time_taken(distance, user_profile["user_car_model"]), user_profile["user_car_model"])
    },
    "Public Transport": {
        "time_taken": round(google_maps_time_taken(distance / 2, "Public Bus") + google_maps_time_taken(distance / 2, "MRT"), 2),
        "emissions": round(emissions_per_trip(google_maps_time_taken(distance / 2, "Public Bus"), "Public Bus") + emissions_per_trip(google_maps_time_taken(distance / 2, "MRT"), "MRT"), 2)
    },
    # assumes that for each public transport journey, half is by bus, the other half is by MRT
    "Walking": {
        "time_taken": google_maps_time_taken(distance, "Walking"),
        "emissions": 0
    },
    "Bicycle": {
        "time_taken": google_maps_time_taken(distance, "Bicycle"),
        "emissions": 0
    }
}

In [22]:
# Step 8: Ranking algorithm
route_data_list = []
for key in route_data_dict:
    route_data_dict[key]["name"] = key
    route_data_list.append(route_data_dict[key])
# Converts the nested dictionary into a list (a variable type which can be ordered)

sorted_route_data_list = sorted(route_data_list, key=itemgetter("emissions", "time_taken"))
# Sorts the list of different possible routes in descending order by emissions, then by time taken, such that tie-breakers in emissions will be broken by comparing the time taken
shortest_time_taken = sorted(route_data_list, key=itemgetter("time_taken"))[0]["time_taken"]

for route in sorted_route_data_list[:]:
    if route["time_taken"] - shortest_time_taken > 1:
        sorted_route_data_list.remove(route)
        sorted_route_data_list.append(route)
# To ensure that the algorithm does not keep recommending walking for an absurdly long period of time, our sorting algorithm features this component where if the route's time taken is longer than the time taken by the fastest route by an hour or more, the algorithm will demote the ranking of that route to the end of the list.
# Because of the earlier sorting by both emissions and time taken, the bicycle route will always be recommended over the walking route due to its faster speed, which is what we want.

In [23]:
# Step 9: Output interface showing the ranking of the different routes
print(f"""
[APP OUTPUT INTERFACE]
Starting Location: {location_1}
Destination: {location_2}
Distance: {distance} km
---------------------""")
for n in range(0,len(sorted_route_data_list)):
    print(f"""Rank {n + 1}: {sorted_route_data_list[n]["name"]}
Est. Duration: {sorted_route_data_list[n]["time_taken"]} hrs
Emissions: {sorted_route_data_list[n]["emissions"]} kg of CO2
""")


[APP OUTPUT INTERFACE]
Starting Location: Geylang
Destination: Woodlands
Distance: 24.65 km
---------------------
Rank 1: Public Transport
Est. Duration: 0.66 hrs
Emissions: 1.0 kg of CO2

Rank 2: Car
Est. Duration: 0.49 hrs
Emissions: 4.9 kg of CO2

Rank 3: Bicycle
Est. Duration: 1.64 hrs
Emissions: 0 kg of CO2

Rank 4: Walking
Est. Duration: 4.93 hrs
Emissions: 0 kg of CO2



In [24]:
# Step 10: Prompting users to choose the route and subtracting the quota from carbon emissions left for the week.
user_choice_index = int(input("Which route will you take? Only key in a single number from '1' - '4': ")) - 1
user_profile["CO2_quota_remaining"] -= sorted_route_data_list[user_choice_index]["emissions"]
print(f"""
The amount of carbon emissions you have left for the week is: {user_profile["CO2_quota_remaining"]} kg out of {user_profile["weekly_CO2_quota_set"]}
""")


Which route will you take? Only key in a single number from '1' - '4': 1

The amount of carbon emissions you have left for the week is: 49.0 kg out of 50

